## Setup Toolhive Operator in OpenShift
**Notes**:
* Skip this step if you already installed the operator following the instructions in the [previous blog post](https://developers.redhat.com/articles/2025/10/01/how-deploy-mcp-servers-openshift-using-toolhive#the_toolhive_operator_on_openshift) of this series.
* If this is the case, ensure to run the latest images for the [operator](https://github.com/stacklok/toolhive/pkgs/container/toolhive%2Foperator/versions?filters%5Bversion_type%5D=tagged) and the [registry server](https://github.com/stacklok/toolhive-registry-server/pkgs/container/thv-registry-api/versions?filters%5Bversion_type%5D=tagged). In case of doubts, use the `latest-ubi` tab for both.


Follow deployment instructions from the [official guide](https://docs.stacklok.com/toolhive/guides-k8s/deploy-operator-helm). from now on, we assume to deploy the operator and all the demo resources in the `toolhive-system` namespace.

### Install the CRDs

In [18]:
!helm upgrade --install toolhive-operator-crds oci://ghcr.io/stacklok/toolhive/toolhive-operator-crds

Pulled: ghcr.io/stacklok/toolhive/toolhive-operator-crds:0.0.88
Digest: sha256:b2f77a114f4dbbd32cc47774224933a12b7eb58188263084b9722f4dd47f6074
Release "toolhive-operator-crds" has been upgraded. Happy Helming!
NAME: toolhive-operator-crds
LAST DEPLOYED: Mon Dec 22 18:01:05 2025
NAMESPACE: toolhive-system
STATUS: deployed
REVISION: 6
TEST SUITE: None


### Install the operator with OpenShift values

In [20]:
!curl -ks -o values-openshift.yaml \
  https://raw.githubusercontent.com/stacklok/toolhive/refs/heads/main/deploy/charts/operator/values-openshift.yaml
!helm upgrade --install toolhive-operator oci://ghcr.io/stacklok/toolhive/toolhive-operator --namespace toolhive-system \
--create-namespace --values ./values-openshift.yaml


Release "toolhive-operator" does not exist. Installing it now.
Pulled: ghcr.io/stacklok/toolhive/toolhive-operator:0.5.17
Digest: sha256:e8e3ceb052b4e63450a087fd0f89b8b9da301859038e028cb9179bc1888318b5
NAME: toolhive-operator
LAST DEPLOYED: Mon Dec 22 18:02:41 2025
NAMESPACE: toolhive-system
STATUS: deployed
REVISION: 1
TEST SUITE: None


Wait until the controller is running

In [21]:
!oc wait pod -l app.kubernetes.io/instance=toolhive-operator --for=condition=Ready --timeout=300s
!oc get pods

pod/toolhive-operator-8675f958b-64fv2 condition met
NAME                                       READY   STATUS    RESTARTS   AGE
cnpg-controller-manager-846579bb9d-cpkdk   1/1     Running   0          11d
demo-db-1                                  1/1     Running   0          18m
toolhive-operator-8675f958b-64fv2          1/1     Running   0          11s


## Setup and deploy Postgres
Prerequisites:
- Install `CloudNativePG` operator. It can also be installed into the `toolhive-system` namespace.

Verify CRDs:

In [22]:
!oc get crd | grep postgresql.cnpg.io

backups.postgresql.cnpg.io                                                         2025-12-05T06:53:38Z
clusterimagecatalogs.postgresql.cnpg.io                                            2025-12-05T06:53:38Z
clusters.postgresql.cnpg.io                                                        2025-12-05T06:53:38Z
databases.postgresql.cnpg.io                                                       2025-12-05T06:53:38Z
failoverquorums.postgresql.cnpg.io                                                 2025-12-05T06:53:38Z
imagecatalogs.postgresql.cnpg.io                                                   2025-12-05T06:53:38Z
poolers.postgresql.cnpg.io                                                         2025-12-05T06:53:38Z
publications.postgresql.cnpg.io                                                    2025-12-05T06:53:38Z
scheduledbackups.postgresql.cnpg.io                                                2025-12-05T06:53:38Z
subscriptions.postgresql.cnpg.io                                

Install the `demo-db` cluster and wait until the database is ready:

In [24]:
!oc apply -f ../manifests/psql-db.yaml
!oc wait pod -l app.kubernetes.io/instance=demo-db --for=condition=Ready --timeout=300s

cluster.postgresql.cnpg.io/demo-db configured
pod/demo-db-1 condition met


## Conclusions

This notebook successfully completed the initial setup required for deploying MCP servers on OpenShift:

✅ **Toolhive Operator**: Installed and configured with OpenShift-specific values in the `toolhive-system` namespace
- CRDs deployed (version 0.0.88)
- Operator deployed (version 0.5.17)
- Controller pod verified as running and ready

✅ **PostgreSQL Database**: Deployed using CloudNativePG operator
- CloudNativePG CRDs verified
- `demo-db` cluster created and running
- Database pod ready for use

**Next Steps**: With the operator and database infrastructure in place, you can now proceed to:
- Deploy the MCP registry server (see notebook [02-deploy-registry.ipynb](./02-deploy-registry.ipynb))
- Deploy and discover MCP servers (see notebook [03-deploy-and-discover-mcp-server.ipynb](./03-deploy-and-discover-mcp-server.ipynb))

All resources are deployed in the `toolhive-system` namespace and ready for the next phase of the demo.
